<a href="https://colab.research.google.com/github/jmcaussade/Arreglo-Incongruencias-Juegos-Jardin/blob/main/Incongruencias_Michel_all_bricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install xlsxwriter
!pip install matplotlib

In [41]:
import pandas as pd
import xlsxwriter
import unicodedata
import matplotlib
import numpy as np  # Import numpy for NaN values

# Functions
def generate_colors(n):
    """Generate n distinct colors."""
    cmap = matplotlib.colormaps['tab20']
    return [matplotlib.colors.to_hex(cmap(i/n)) for i in range(n)]

def normalizar_texto(texto):
    """Normalize text - convert to lowercase and remove accents."""
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode("utf-8")
    return texto

# Read Excel File
archivo_excel = 'Analisis.xlsx'
df = pd.read_excel(archivo_excel, sheet_name='Hoja1')

# Define columns to exclude
excluir_columnas = ["<ID>", "SKU - Chile", "Jefe de Línea", 'Descripción Web', '<Name>', "Marca", "Origen", "<Parent ID>"]

# Generate colors for columns
initial_col_count = max(len(df.columns) - len(excluir_columnas), 1)
colores_columnas = generate_colors(initial_col_count)

# Process DataFrame
for i, col in enumerate(df.columns):
    if col not in excluir_columnas:
        insert_position = df.columns.get_loc(col) + 1
        col_aparece = f"{col} aparece"
        df.insert(insert_position, col_aparece, df.apply(lambda row: normalizar_texto(str(row[col])) in
                                                        normalizar_texto(str(row['<Name>'])) or
                                                        normalizar_texto(str(row[col])) in
                                                        normalizar_texto(str(row['Descripción Web'])), axis=1))

# Calculate Row-wise Error Percentage
boolean_columns = [col for col in df.columns if 'aparece' in col]
df['Error percentage (Row)'] = df[boolean_columns].apply(lambda row: (row.value_counts().get(False, 0) * 100) / len(row), axis=1)

# Create a DataFrame to hold the error percentages by column
error_percentages_col = pd.DataFrame({col: [(df[col].value_counts().get(False, 0) * 100) / len(df[col])] for col in boolean_columns})

# Export to Excel
archivo_salida = 'test_codigo1.xlsx'
writer = pd.ExcelWriter(archivo_salida, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1', index=False)

# Formatting
workbook = writer.book
worksheet = writer.sheets['Sheet1']
format_verdadero = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
format_falso = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})

for col_num, value in enumerate(df.columns.values):
    if value not in excluir_columnas and 'aparece' not in value:
        color_index = (col_num // 2) % len(colores_columnas)
        cell_format = workbook.add_format({'bg_color': colores_columnas[color_index]})
        worksheet.set_column(col_num, col_num, None, cell_format)
    elif 'aparece' in value:
        worksheet.conditional_format(1, col_num, len(df), col_num,
                                     {'type': 'cell',
                                      'criteria': '=',
                                      'value': True,
                                      'format': format_verdadero})
        worksheet.conditional_format(1, col_num, len(df), col_num,
                                     {'type': 'cell',
                                      'criteria': '=',
                                      'value': False,
                                      'format': format_falso})

# Add Error Percentages by Column under the corresponding columns
for i, col in enumerate(boolean_columns):
    for j, value in enumerate(error_percentages_col[col].values):
        worksheet.write(len(df) + 1 + j, i, value if not np.isnan(value) else "NaN")

# Add "Error Percentage by Attribute" in the first cell of the row after the data ends
worksheet.write(len(df) + 1, 0, "Error Percentage by Attribute")


# Close and save Excel writer
writer.close()
print(f"Se ha exportado el DataFrame con formato a '{archivo_salida}'.")


Se ha exportado el DataFrame con formato a 'test_codigo1.xlsx'.


<h1>DISPLAY FILE </h1>


In [42]:
import pandas as pd

# Replace 'your_excel_file.xlsx' with the path to your Excel file
# and 'Sheet1' with the name of the sheet you want to display
file_path = 'test_codigo1.xlsx'
sheet_name = 'Sheet1'

df = pd.read_excel(file_path, sheet_name=sheet_name)

# Display the DataFrame
# You can use df.head() to display the first few rows
# or df to display the entire DataFrame (be cautious with large DataFrames)
df


,SKU - Chile,<Name>,Jefe de Línea,<Parent ID>,Descripción Web,Tipo de cocina,Tipo de cocina aparece,Número de quemadores,Número de quemadores aparece,Modelo,Modelo aparece,Error percentage (Row)
0,1259636,1259636 - PE COC 4P 62L 6EA48 MAIGAS IND,42 - PM Matias Barros,10003690.0,Cocina a Gas 4 Platos Silver 1259636-1,Cocina a Gas,1.0,4.0,1.0,1259636-1,1.0,0.0
1,1259644,1259644 - PE COC 4P 84L 6EA50 MAIGAS IND,42 - PM Matias Barros,10003690.0,Cocina a Gas 4 Platos Silver 1259644-1,Cocina a Gas,1.0,4.0,1.0,1259644-1,1.0,0.0
2,1612719,1612719 - COCINA 4P 66L F2525T FENSA,77 - Daniela Barriga,10003690.0,Cocina a Gas 4 Platos Gris F2525T,Cocina a Gas,1.0,4.0,1.0,F2525T,1.0,0.0
3,1746952,1746952 - PE COCINA 4P 57L PRO-Q4-GL,42 - PM Matias Barros,10003690.0,Cocina a Gas 4 Platos Silver Pro-Q4-GL,Cocina a Gas,1.0,4.0,1.0,Pro-Q4-GL,1.0,0.0
4,2144638,2144638 - COCINA 4P 68L CH-9050SIL SIND,77 - Daniela Barriga,10003690.0,Cocina a Gas 4 Platos Silver CH-9050SIL,Cocina a Gas,1.0,4.0,1.0,CH-9050SIL,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
69,7454767,7454767 - PE COCINA GAS 4 QUEMADORES NGO,42 - PM Matias Barros,10003690.0,Cocina a Gas 4 Platos Negra NX24BG45411VZS,Cocina a Gas,1.0,4.0,1.0,NX24BG45411VZS,1.0,0.0
70,7454554,7454554 - PE COCINA GAS 4 QUEMADORES SLV,42 - PM Matias Barros,10003690.0,Cocina a Gas 4 Platos Silver NX24BG57413SZS,Cocina a Gas,1.0,4.0,1.0,NX24BG57413SZS,1.0,0.0
71,7461402,7461402 - PE COCINA MIDEA MCG-4QPI24NS,29 - PM Diego Cusso Saavedra,10003690.0,Cocina a Gas 4 Platos Silver MCG-4QPI24NS,Cocina a Gas,1.0,4.0,1.0,MCG-4QPI24NS,1.0,0.0
72,7469918,7469918 - PE COCINA MIDEA MCG-4QSG20NSBK,42 - PM Matias Barros,10003690.0,Cocina a Gas 4 Platos Negro MCG-4QSG20NSBK,Cocina a Gas,1.0,4.0,1.0,MCG-4QSG20NSBK,1.0,0.0
